In [1]:
!java -version

openjdk version "11.0.19" 2023-04-18
OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)


In [2]:
!pip install pyspark

In [3]:
!pip install findspark

In [4]:
import findspark
findspark.init()
findspark.find()

'/usr/local/lib/python3.10/dist-packages/pyspark'

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
emptyRDD = spark.sparkContext.emptyRDD()
print(emptyRDD)

EmptyRDD[0] at emptyRDD at NativeMethodAccessorImpl.java:0


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, collect_list

In [7]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
file_path = '/content/drive/MyDrive/hackrx/'
customer = pd.DataFrame(pd.read_csv(file_path+"reviews.csv"))
cat_df = pd.DataFrame(pd.read_csv(file_path+"categories.csv"))
customer = customer.head(10000)
cat_df = cat_df.head(10000)
modified_df = cat_df.copy()
modified_df['category'] = modified_df['category'].str.replace(r'\[.*?\]', '').str.lstrip().str.split('|')
modified_df
temp=modified_df.groupby('ASIN').sum().reset_index()
temp.head()
# temp['category'][0],modified_df['category'][0],modified_df['category'][1],modified_df['category'][2]
temp['category']= temp['category'].apply(lambda x:list(set(filter(None,x))))
temp['category'] = temp['category'].apply(lambda x: [word.lower() for word in x])
temp.columns
temp['category'][0],modified_df['category'][0],modified_df['category'][1]
reviews_df = pd.DataFrame(pd.read_csv("/content/drive/MyDrive/hackrx/reviews.csv"))
temp.columns
reviews_df.columns
merged_df = pd.merge(temp,reviews_df,left_on='ASIN',right_on='ASIN')
final_df = merged_df.drop('ASIN',axis=1)
final_df = final_df.groupby('customer id')['category'].sum().reset_index()
merged_df.drop('ASIN',axis=1,inplace=True)
customer_df = merged_df.groupby('customer id')['category'].sum().reset_index()
customer_df

<ipython-input-9-d90efd51a1c0>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  modified_df['category'] = modified_df['category'].str.replace(r'\[.*?\]', '').str.lstrip().str.split('|')


,customer id,category
0,A1001URKW36W4Z,"[a-z, parenting, general, by occasion, subject..."
1,A100DMMBKT8HO5,"[subjects, history, books, aviation, specialty..."
2,A100T19LDH0GGN,"[styles, general, dance & dj, music]"
3,A100TW8FZECWD6,"[( j ), subjects, a-z, books, johnson, histori..."
4,A100ZYQ4ENTZ57,"[new age, international, amazon.com stores, ge..."
...,...,...
21609,AZYFIHUY6BCBQ,"[people & places, science, general, counting, ..."
21610,AZYFJ8OO3B1UH,"[subjects, books, general, contemporary, liter..."
21611,AZYG4QMGSL4ZX,"[general, contemporary country, country, music..."
21612,AZYM1KLRKD2GH,"[subjects, a-z, books, mystery & thrillers, wo..."


In [10]:
reviews = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/hackrx/reviews.csv'))

In [11]:
reviews = reviews.head(10000)

In [12]:
top10000customer_list = list(reviews['customer id'])

In [13]:
'''
customer data to be implemented on
'''
customer_df = customer_df[customer_df['customer id'].isin(top10000customer_list)]

In [14]:
customer_df.reset_index(inplace=True)

In [15]:
customer_df.drop('index',axis=1,inplace=True)

<ipython-input-15-03a39ca524a3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  customer_df.drop('index',axis=1,inplace=True)


In [16]:
customer_df

,customer id,category
0,A100DMMBKT8HO5,"[subjects, history, books, aviation, specialty..."
1,A101OA1WQ56H5I,"[alternative rock, alternative metal, hard roc..."
2,A101VYVQR1Y70U,"[alternative metal, hard rock & metal, general..."
3,A103U0Q3IKSXHE,"[subjects, religion & spirituality, books, env..."
4,A104W9PL1OO5JN,"[subjects, a-z, books, mystery & thrillers, sp..."
...,...,...
8294,AZVRZHRWM2OVA,"[subjects, a-z, books, mystery & thrillers, sp..."
8295,AZWF64HVI67SI,"[subjects, books, practical politics, general,..."
8296,AZX4S8FYUPCQG,"[subjects, a-z, books, general, hornby, romanc..."
8297,AZXC3VFGAGXYH,"[new age, international, amazon.com stores, ge..."


In [17]:
'''
no duplicated items
'''
import networkx as nx

user_graph = nx.Graph()

for i in range(len(customer_df)):
  user_graph.add_node(customer_df['customer id'][i],feature = customer_df["category"][i])

In [18]:
temp = (user_graph.nodes['AZYM1KLRKD2GH'])

In [19]:
print(len(user_graph.nodes()))

8299


In [20]:
!pip install node2vec

In [21]:
from node2vec import Node2Vec

node2vec = Node2Vec(user_graph, dimensions=64, walk_length=10, num_walks=20, workers=4)

# Generate the node embeddings
model = node2vec.fit(window=10, min_count=1)

Computing transition probabilities:   0%|          | 0/8299 [00:00<?, ?it/s]

In [22]:
'''
make graph based on the similarity between the customers
'''
node_embeddings = {str(node): model.wv[str(node)] for node in user_graph.nodes}

In [23]:
def compute_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

In [24]:
import tqdm

In [25]:
import time
import itertools

In [26]:
all_node_pairs = []
all_node_pairs = list(itertools.combinations(user_graph.nodes, 2))
print(len(all_node_pairs))
for u, v in tqdm.tqdm(all_node_pairs):
  if not user_graph.has_edge(u, v):
    similarity = compute_similarity(node_embeddings[str(u)], node_embeddings[str(v)])
    if similarity > 0.75:
      user_graph.add_edge(u, v)


34432551


100%|██████████| 34432551/34432551 [11:58<00:00, 47947.61it/s]


In [29]:
import json

In [31]:
file_path  = "/content/drive/MyDrive/user_user_graph.json"
data = nx.node_link_data(user_graph)
with open(file_path, "w") as f:
    json.dump(data, f)

# After the graph is done

In [32]:
with open(file_path, "r") as f:
    data = json.load(f)

# Create the NetworkX graph from the JSON data
G = nx.node_link_graph(data)

In [35]:
'''
i want each nodes containing the products bought so far
'''
reviews = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/hackrx/reviews.csv'))

In [42]:
reviews

,ASIN,customer id,rating,votes,helpful
0,0827229534,A2JW67OY8U6HHK,5,10,9
1,0827229534,A2VE83MZF98ITY,5,6,5
2,0738700797,A11NCO6YTE4BTJ,5,5,4
3,0738700797,A9CQ3PLRNIR83,4,5,5
4,0738700797,A13SG9ACZ9O5IM,5,8,8
...,...,...,...,...,...
1289691,B00000I4LU,A2RV7R0OCRDRWX,5,1,1
1289692,B00000I4LU,ATVPDKIKX0DER,5,0,0
1289693,B00000I4LU,ATVPDKIKX0DER,5,0,0
1289694,B00000I4LU,ATVPDKIKX0DER,5,0,0


In [37]:
list_of_items = reviews.drop(['rating','votes','helpful'],axis=1)

In [45]:
grouped_df = list_of_items.groupby('customer id')['ASIN'].agg(list).reset_index()

In [47]:
final_df = grouped_df[grouped_df['customer id'].isin(top10000customer_list)]

In [51]:
filtered_rows = final_df[final_df['ASIN'].apply(lambda x: len(x) > 1)]

           customer id                                               ASIN
191     A101VYVQR1Y70U  [B000068GA4, B00005TPL2, 1578590477, 158240272...
406     A103U0Q3IKSXHE  [0824518659, 0767913825, 0800631307, 006017540...
528     A104W9PL1OO5JN                           [0385504209, 0807220280]
570     A105B8SVTVDBHU  [B0000296JC, B00007L4Q3, B0000007RS, B0000007R...
647      A106016KSI0YQ  [0736649417, 038548948X, 0375757260, 080213797...
...                ...                                                ...
524368   AZU9MFWBU4B68   [0385504209, 0966613058, 0679757031, 0679757031]
524405   AZUKKLJSA33BI   [088404291X, 0884042820, 0884046818, 0884046818]
524525   AZVH70JMJ2IHX  [B000003ONH, B00005NTOH, B00005MKMD, B000002B8...
524546   AZVRZHRWM2OVA                           [0385504209, 0694519405]
524618   AZWF64HVI67SI                           [0767905288, 0553712438]

[4547 rows x 2 columns]


In [59]:
user_items_dict = final_df.set_index('customer id')['ASIN'].to_dict()

In [63]:
for node in G.nodes:
  user_id = node
  if user_id in user_items_dict:
    item = user_items_dict[user_id]
    G.nodes[node]['item'] = item
  else:
    G.nodes[node]['item'] = []

In [65]:
file_path  = "/content/drive/MyDrive/user_user_graph.json"
data = nx.node_link_data(G)
with open(file_path, "w") as f:
    json.dump(data, f)